<a href="https://colab.research.google.com/github/1040mxg/5334project/blob/main/5334proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

[Blog homepage](https://1040mxg.github.io/blog/) || [View on Github](https://github.com/1040mxg/blog/blob/gh-pages/_posts/2021-04-27-nbc.md)

The goal of this assignment is to learn about Naive Bayes Classifiers by building a classifier and training and testing it on an IMDB dataset with the goal of predicting whether a given review's sentiment is "Positive" or "Negative".

**Naive Bayes** is a simple, fast, and accurate algorithm that works particularly well with natural language processing (NLP), or text classification. In my tests, I used the **Multinomial Naive Bayes** algorithm. It works by taking advantage of probability theory and Bayes' Theorem to classify text. For each piece of input text, the probability of each possible class is calculated and the final classification made based on highest probability.


In [ ]:
pip install --user -U nltk

Requirement already up-to-date: nltk in /root/.local/lib/python3.7/site-packages (3.6.2)


# Preprocessing

Import IMDB dataset and prepare a data array for later graphing.

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import ngrams
from collections import Counter

stopwords = stopwords.words('english')

##import data file and print some basic stats
file = "tweets_xh_hk.txt"
file2 = "tweets_npr_hk.txt"
xhData = []
nprData = []

with open(file, 'r') as f:
  for row in f:
    xhData.append(row)

with open(file2, 'r') as f:
  for row in f:
    nprData.append(row)

data = []

for i in range(len(xhData)):
  if len(xhData[i]) > 25:
    data.append([xhData[i], '0'])

for i in range(len(nprData)):
  if len(nprData[i]) > 25:
    data.append([nprData[i], '1'])

X = []
y = []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Divide the dataset into train, dev, and test sets. First, the data is shuffled and then divided manually using index numbers. To start, the data was divided in an 80/20 ratio for the initial train and test set. Then, the training set was further divided again in an 80/20, train/dev ratio.

The final sizes for each dataset are as follows:
*   train = 640 items
*   dev = 160 items
*   tets = 200 items

In [ ]:
import random

print("XH:", len(xhData))
print("NPR:", len(nprData))

random.seed(1)
random.shuffle(data)
dataLen = len(data)

train = []
dev = []
test =[]

#test = 680items

print(dataLen)
testNum = 680
trainNum = 2182
devNum = 545

for i in range(trainNum):
  train.append(data[i])

for i in range(trainNum, trainNum+devNum):
  dev.append(data[i])

for i in range(dataLen-testNum, dataLen):
  test.append(data[i])

print("Train Data: %d items" %(len(train)))
print("Dev Data: %d items" %(len(dev)))
print("Test Data: %d items" %(len(test)))

XH: 2893
NPR: 514
3232
Train Data: 2182 items
Dev Data: 545 items
Test Data: 680 items


# Training

Using regular expressions, the training set of 640 items (lines) is split into two lists based on positive/negative sentiment.

In [ ]:
import re

#pos = re.search("1$")
#neg = re.search("0$")

#split trainset into pos + negative using regular expressions
def splitPosNeg(data):
  posStr = []
  negStr = []
  for i in range(len(data)):
    if data[i][-1] == '1':
      posStr.append(data[i][0])
    elif data[i][-1] == '0':
      negStr.append(data[i][0])
  return posStr, negStr

def printPosNeg(posStr, negStr):
  print("Positive Reviews: %d items" %(len(posStr)))
  print("---------------Sample---------------")
  for i in range(5):
    if(len(posStr[i])>80):
      print("%s... 1" %posStr[i][0:80])
    else:
      print(posStr[i], end='')

  print("\n")

  print("Negative Reviews: %d items" %(len(negStr)))
  print("---------------Sample---------------")
  for i in range(5):
    if(len(negStr[i])>80):
      print("%s... 0" %negStr[i][0:80])
    else:
      print(negStr[i], end='')

posStr, negStr = splitPosNeg(train)
printPosNeg(posStr, negStr)

Positive Reviews: 235 items
---------------Sample---------------
Beijing has enacted a controversial new security law in Hong Kong. It criminaliz... 1
Hong Kong Police Use Tear Gas On Large Pro-Democracy Protest n.pr/1uTEaeZ
For months, Hong Kong has been wracked by protests. What are the demonstrators f... 1
President Trump said he supports pro-democracy demonstrators in Hong Kong — but ... 1
NPRs @EmilyZFeng live in Hong Kong: twitter.com/EmilyZFeng/sta…


Negative Reviews: 1947 items
---------------Sample---------------
Welcome to @Disneyland in Hong Kong. You gotta plenty to enjoy in this sparkling... 0
U.S. Congress' passage of the so-called Hong Kong Human Rights and Democracy Act... 0
"#UPDATE: 52 countries welcome China's adoption of law on safeguarding national ... 0
Check out our #AsiaAlbum series here xhne.ws/uKix6 https://t.co/M22mzxvNkC"
The Committee for Safeguarding National Security of the Hong Kong Special Admini... 0


The positive/negative review lists are further processed into dictionaries of vocabularies.

First, each line in the lists was split into individual words and added to a temporary dictionary. This temporary dictionary was then trimmed to include only words that occur more than 3 times, and are greater than 3 characters in length to help filter out articles like "a", "an", as well as the classifying 1/0. The reasoning behind this was to focus on words that contribute to a sentiment, without filler words in the data. This unfortunately leaves in filler words like "the" or "and", but leaves in potentially important words like "bad".

In [ ]:
#trainset vocab list + split into pos/neg
def splitWord(strings, tempDict):
  words = 0
  for line in strings:
    line = line.lower()
    temp = re.findall(r'\w+', line)
    for word in temp:
      words+=1
      if word in tempDict:
        tempDict[word] +=1
      else:
        tempDict[word] = 1
  #print("Total Word Count: ", words)

"""
remove all occurrences < 3, all words 2 characters or less to take care of articles like "a", "an", as well as the classifying 1/0
"""
def trim(strings, tempDict):
  keys = list(tempDict.keys())
  vals = list(tempDict.values())
  newDict = dict()
  count = 0
  for i in range(len(vals)):
    k = keys[i]
    if vals[i] >= 3 and k not in stopwords:
      newDict[k] = vals[i]
      count+=vals[i]
  return newDict, count

def makeVocab(strings):
  tempDict = dict()
  splitWord(strings, tempDict)
  newDict, count = trim(strings, tempDict)
  return newDict, count

print("---------------Positive Vocabulary---------------")
posVocab, posCount = makeVocab(posStr)
print("Words After Trimming: ", posCount)
print("Unique Words: %d\n" %len(posVocab))

print("---------------Negative Vocabulary---------------")
negVocab, negCount = makeVocab(negStr)
print("Words After Trimming: ", negCount)
print("Unique Words: %d\n" %len(negVocab))

print("---------------All Words---------------")
allVocab = {**posVocab, **negVocab}
allCount = posCount+negCount
print("Total Word Count: ", allCount)
print("Unique Words: ", len(allVocab))

# print("Positive:", posVocab)
# print("Negative:", negVocab)
# print("All:", allVocab)  

---------------Positive Vocabulary---------------
Words After Trimming:  1883
Unique Words: 210

---------------Negative Vocabulary---------------
Words After Trimming:  29445
Unique Words: 1802

---------------All Words---------------
Total Word Count:  31328
Unique Words:  1856


P(positive) and P(negative) were calculated by dividing the count of each by the total (trimmed) word count.

Individual P(word|sentiment) were calculated using (#word in class)/(#total word count in sentiment class). A general P(word) was also calculated using (#word/#total word count). All three were stored in new dictionaries.

In [ ]:
#get probabilities for P(positive), P(negative) based on trimmed word counts
allCount = posCount+negCount
print("---------------Probabilities---------------")
print("P(positive) = %d/%d = %.5f" %(posCount, allCount, posCount/allCount))
print("P(negative) = %d/%d = %.5f\n" %(negCount, allCount, negCount/allCount))

#get individual P(word|class) and store in dictionary
def getProbs(Dict, PCount):
  keys = list(Dict.keys())
  vals = list(Dict.values())
  newDict = dict()
  for i in range(len(vals)):
    k = keys[i]
    p = vals[i]/PCount
    newDict[k] = p
  return newDict

posProb = getProbs(posVocab, posCount)
negProb = getProbs(negVocab, negCount)
allProb = getProbs(allVocab, allCount)

print("---------------P(word) Sample---------------")
print({k: allProb[k] for k in list(allProb)[:5]}, "\n")

print("---------------P(word|positive) Sample---------------")
print({k: posProb[k] for k in list(posProb)[:5]}, "\n")

print("---------------P(word|negative) Sample---------------")
print({k: negProb[k] for k in list(negProb)[:5]})

---------------Probabilities---------------
P(positive) = 1883/31328 = 0.06011
P(negative) = 29445/31328 = 0.93989

---------------P(word) Sample---------------
{'beijing': 0.0006064862104187947, 'controversial': 0.00015960163432073544, 'new': 0.0031601123595505617, 'security': 0.005905260469867212, 'law': 0.00450076608784474} 

---------------P(word|positive) Sample---------------
{'beijing': 0.007434944237918215, 'controversial': 0.0026553372278279343, 'new': 0.0053106744556558685, 'security': 0.004248539564524694, 'law': 0.0053106744556558685} 

---------------P(word|negative) Sample---------------
{'welcome': 0.0003056546102903719, 'disneyland': 0.00020376974019358125, 'hong': 0.06255731023942944, 'kong': 0.06262523348616064, 'enjoy': 0.00033961623365596876}


# Testing Model with Dev Set

The dev set of 160 items was used for the following:
*   General accuracy testing of the fitted training model
*   Five-fold cross validation to test the general algorithm.
*   Accuracy testing with Laplace smoothing


**General Accuracy Testing**:

Line by line, the process was:
1.   Split into individual words
2.   Determine and save the actual sentiment class
3.   Remove words ≤ 2 characters in order to be consistent with the training data and prevent unnecessary zeroes. 
4.   Calculate a P(positive), P(negative) using the vocabulary and probability dictionaries created earlier.
5.   Compare P(positive) and P(negative), taking the higher value as the predicted class.
6.   Compared the predicted class with the actual class, and add on accordingly to a correct/wrong count.

The final prediction accuracy was then calculated.

In [ ]:
# print("---------------Sample Dev Set---------------")
# for i in range(7):
#   print(dev[i], end='')

from random import randrange

def actualClass(line):
  if line[-1] == '1':
    return 1
  elif line[-1] == '0':
    return 0

def splitTrim(line):
  words = []
  temp = re.findall(r'\w+', line[0])
  #print(temp)
  for i in temp:
    if len(i)>2:
      words.append(i)
  #print(words)
  return words

def naiveBayes(line):
  words = splitTrim(line)
  p_pos = 1
  p_neg = 1
  for word in words:
    if word in posVocab:
      p_pos = posProb[word]*p_pos
    else:
      p_pos = p_pos*0
    if word in negVocab:
      p_neg = negProb[word]*p_neg
    else:
      p_neg = p_neg*0
  if p_pos > p_neg:
    pred = 1
  elif p_neg > p_pos:
    pred = 0
  else:
    pred = 2
  return pred

def predict(data):
  correct = 0
  total = 0
  for line in data:
    #print(line)
    actual = actualClass(line)
    pred = naiveBayes(line)
    if pred == actual:
      correct+=1
      total+=1
    else:
      total+=1
  acc = correct/total
  return acc, correct, total

##train on train set just for comparison
accu, correct, total = predict(train)
X.append("Train")
y.append(accu*100)

accu, correct, total = predict(dev)
print("Accuracy: %d/%d = %.2f" %(correct, total, accu*100),"%")
X.append("Dev")
y.append(accu*100)

Accuracy: 0/545 = 0.00 %


**Five-Fold Cross Validation**

This tests the algorithm using cross validation. The process is as follows:
1.   Split the data into 5 groups
2.   For each group, use it as a test set with the remaining groups as training sets
3.   Fit a model on the training set and evaluate on the test set
4.   Repeat the above until all samples have had a turn as a test set
5.   Find the mean accuracy

In [ ]:
from random import randrange

random.seed(1)

def x_naiveBayes(line):
  words = splitTrim(line)
  p_pos = 1
  p_neg = 1
  for word in words:
    if word in posVocab:
      p_pos = posProb[word]*p_pos
    else:
      p_pos = p_pos*0
    if word in negVocab:
      p_neg = negProb[word]*p_neg
    else:
      p_neg = p_neg*0
  if p_pos > p_neg:
    pred = 1
  elif p_neg > p_pos:
    pred = 0
  else:
    pred = 2
  return pred

def x_predict(data):
  correct = 0
  total = 0
  for line in data:
    #print(line)
    actual = actualClass(line)
    pred = x_naiveBayes(line)
    if pred == actual:
      correct+=1
      total+=1
    else:
      total+=1
  acc = correct/total
  return acc, correct, total

#split dataset into 5 folds
def crossValsplit(dataset):
  folds = 5
  data_split = list()
  data_copy = list(dataset)
  fold_size = int(len(dataset)/folds)
  for i in range(folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(data_copy))
      fold.append(data_copy.pop(index))
    data_split.append(fold)
  return data_split

def evaluate(data):
  folds = crossValsplit(data)
  scores = list()
  for fold in folds:
    train = list(folds)
    train.remove(fold)
    train = sum(train, [])
    test = list()
    for row in fold:
      test.append(row)
    #create new dictionaries based on fold
    x_posStr, x_negStr = splitPosNeg(train)
    x_posVocab, x_posCount = makeVocab(x_posStr)
    x_negVocab, x_negCount = makeVocab(x_negStr)
    x_allVocab = {**x_posVocab, **x_negVocab}
    x_allCount = x_posCount+x_negCount
    x_posProb = getProbs(x_posVocab, x_posCount)
    x_negProb = getProbs(x_negVocab, x_negCount)
    x_allProb = getProbs(x_allVocab, x_allCount)
    acc, correct, total = x_predict(test)
    acc = acc*100
    scores.append(acc)
  print("Scores: %s" %scores)
  print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
  X.append("Cross-Val")
  y.append(sum(scores)/float(len(scores)))
    
evaluate(dev)

Scores: [0.0, 0.0, 0.0, 0.0, 0.0]
Mean Accuracy: 0.000%


Based on the mean accuracy from cross validation, the current algorithm is not very good and needs refinement before it can be tested against the test dataset.

# Experiments

**Laplace Smoothing**

One major issue that Naive Bayes faces in text classification is missing data. When a word does not appear in a class, making P(word|class) = 0, that probability gets multiplied out to all the other probabilities in determining P(class), making P(class) = 0.

Laplace smoothing is a method of combatting this issue. The process is:
1.   Add 1 to every count so P(word|class) will always > 0.
2.   Balance this by adding the number of possible words to the divisor so the result will never < 1.

Smoothing is applied to the training set of 640 below, adding 1 to every unique word count and total words to #positive and #negative.

In [ ]:
def trim(strings, tempDict):
  keys = list(tempDict.keys())
  vals = list(tempDict.values())
  newDict = dict()
  count = 0
  for i in range(len(vals)):
    k = keys[i]
    if vals[i] >= 3 and len(k) > 2:
      #smoothing: unique count + 1
      newDict[k] = vals[i]+1
      count+=vals[i]
  return newDict, count

def makeVocab(strings):
  tempDict = dict()
  splitWord(strings, tempDict)
  newDict, count = trim(strings, tempDict)
  return newDict, count

posVocab, posCount = makeVocab(posStr)
negVocab, negCount = makeVocab(negStr)

allVocab = {**posVocab, **negVocab}

#laplace smoothing
posCount+=len(allVocab)
negCount+=len(allVocab)
allCount = posCount+negCount
#for test words of P=0
noPos = 1/posCount
noNeg = 1/negCount

def getProbs(Dict, PCount):
  keys = list(Dict.keys())
  vals = list(Dict.values())
  newDict = dict()
  for i in range(len(vals)):
    k = keys[i]
    p = vals[i]/PCount
    newDict[k] = p
  return newDict

posProb = getProbs(posVocab, posCount)
negProb = getProbs(negVocab, negCount)
allProb = getProbs(allVocab, allCount)

from random import randrange

def actualClass(line):
  if line[-1] == '1':
    return 1
  elif line[-1] == '0':
    return 0

def splitTrim(line):
  words = []
  temp = re.findall(r'\w+', line[0])
  #print(temp)
  for i in temp:
    if len(i)>2:
      words.append(i)
  #print(words)
  return words

def naiveBayes(line):
  words = splitTrim(line)
  p_pos = 1
  p_neg = 1
  for word in words:
    if word in posVocab:
      p_pos = posProb[word]*p_pos
    else:
      p_pos = p_pos*noPos
    if word in negVocab:
      p_neg = negProb[word]*p_neg
    else:
      p_neg = p_neg*noNeg
  if p_pos > p_neg:
    pred = 1
  elif p_neg > p_pos:
    pred = 0
  return pred

def predict(data):
  correct = 0
  total = 0
  for line in data:
    #print(line)
    actual = actualClass(line)
    pred = naiveBayes(line)
    if pred == actual:
      correct+=1
      total+=1
    else:
      total+=1
  acc = correct/total
  return acc, correct, total

acc, correct, total = predict(dev)
acc = acc*100
print("Accuracy (Dev set with Smoothing): %d/%d = %.2f" %(correct, total, acc),"%")
X.append("Dev+\nSmoothing")
y.append(acc)

Accuracy (Dev set with Smoothing): 106/545 = 19.45 %


Applying smoothing dramatically improved the predictions' accuracy.

**Top Words**

Next, I will derive the top 10 words for predicting class, ie. the top 10 each of P(Positive|word) and P(Negative|word). Given that:

$P(class|word)=\frac{P(word|class)P(class)}{P(word)}$

and I already have the values for P(word), P(class), and P(word|class), this will be a simple matter of finding the top 10 values.

In [ ]:
"""
Find the top ten P(class|word) for each class
"""



# def topTen(classDict, classCount):
#   words = []
#   probs = []
#   #keys = word, vals = P(word|class)
#   keys = list(classDict.keys())
#   vals = list(classDict.values())
#   pClass = classCount/allCount
#   for i in range(len(keys)):
#     pWord = allProb[(keys[i])]
#     p = (vals[i]*pClass)/pWord
#     words.append(keys[i])
#     probs.append(p)
#   top = sorted(range(len(probs)), key=lambda i: probs[i])[-10:]
#   topWords = []
#   for i in top:
#     topWords.append(words[i])
#   return topWords

# topPos = topTen(posProb, posCount)
# topNeg = topTen(negProb, negCount)

# print("Top Positive Words:")
# print(topPos)
# print("\nTop Negative Words:")
# print(topNeg)



'\nFind the top ten P(class|word) for each class\n'

# Testing

Finally, I will calculate the final accuracy. Based on the experiments above, using Laplace smoothing by itself provided the best accuracy, so that will be the approach I use here.

In [ ]:
from random import randrange

def actualClass(line):
  if line[-1] == '1':
    return 1
  elif line[-1] == '0':
    return 0

def splitTrim(line):
  words = []
  temp = re.findall(r'\w+', line[0])
  #print(temp)
  for i in temp:
    if len(i)>2:
      words.append(i)
  #print(words)
  return words

#using laplace where noPos and noNeg = 1/classCount
def naiveBayes(line):
  words = splitTrim(line)
  p_pos = 1
  p_neg = 1
  for word in words:
    if word in posVocab:
      p_pos = posProb[word]*p_pos
    else:
      p_pos = p_pos*noPos
    if word in negVocab:
      p_neg = negProb[word]*p_neg
    else:
      p_neg = p_neg*noNeg
  if p_pos > p_neg:
    pred = 1
  elif p_neg > p_pos:
    pred = 0
  return pred

def predict(data):
  correct = 0
  total = 0
  for line in data:
    #print(line)
    actual = actualClass(line)
    pred = naiveBayes(line)
    if pred == actual:
      correct+=1
      total+=1
    else:
      total+=1
  acc = correct/total
  return acc, correct, total

acc, correct, total = predict(test)
acc = acc*100
print("Accuracy: %d/%d = %.2f" %(correct, total, acc),"%")
X.append("Test")
y.append(acc)

Accuracy: 129/680 = 18.97 %


This has resulted in the highest accuracy of all the tests at 76.50%

In [ ]:
with open("hktwmacao.txt", "r") as file:
  text = file.read()



In [ ]:
import string
import collections

# get rid of all the XML markup
text = re.sub('<.*>','',text)

# get rid of the "ENDOFARTICLE." text
text = re.sub('ENDOFARTICLE.','',text)

# get rid of punctuation (except periods!)
punctuation = "[" + re.sub("\"","",string.punctuation) + "]"
text = re.sub(punctuation, "", text)

tokenized = text.split()

# and get a list of all the bi-grams
bigrams = ngrams(tokenized, 2)
yongrams = ngrams(tokenized, 4)
gograms = ngrams(tokenized, 5)

top2 = collections.Counter(ngrams(tokenized, 2)).most_common(10)
top4 = collections.Counter(ngrams(tokenized, 4)).most_common(10)
top5 = collections.Counter(ngrams(tokenized, 5)).most_common(10)

top5


[(('Hong', 'Kong', 'Special', 'Administrative', 'Region'), 239),
 (('Kong', 'Special', 'Administrative', 'Region', 'HKSAR'), 191),
 (('the', 'Hong', 'Kong', 'Special', 'Administrative'), 163),
 (('of', 'the', 'Hong', 'Kong', 'Special'), 105),
 (('the', 'Standing', 'Committee', 'of', 'the'), 85),
 (('Special', 'Administrative', 'Region', 'HKSAR', 'government'), 70),
 (('of', 'the', 'National', 'Peoples', 'Congress'), 61),
 (('Hong', 'Kongs', 'Center', 'for', 'Health'), 60),
 (('Kongs', 'Center', 'for', 'Health', 'Protection'), 60),
 (('Standing', 'Committee', 'of', 'the', 'National'), 57)]

# Analysis

In all the tests, only the ones that made use of Laplace smoothing had accuracies > 50%. This clearly shows one of the limitations of this particular algorithm: unknown and unaccounted-for data will have their probabilities of zero multiply out, rendering many classification attempts useless. However, by simply incorporating smoothing, this issue is largely resolved.

**Future Possibilities**

In my tests, I used certain trimming methods on the datasets (frequency > 3, word length > 2 characters). Other possible tests could experiment with different values of the two. Alternatively, instead of setting a frequency/length minimum, I could just import a list of stopwords such as that available from Natural Language Toolkit's [corpus](https://pythonprogramming.net/stop-words-nltk-tutorial/) and remove any stopwords from the datasets.

# References

*   Bruno Stecanella, "A practical explanation of a Naive Bayes classifier", https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/
*   Jason Brownlee, "Naive Bayes Classifier From Scratch in Python
", https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/


